# Algorithm 19: Noise Level Embedding (AlphaFold3)

Sinusoidal embedding for noise level in diffusion model, similar to positional encoding.

## Source Code Location
- **File**: `AF3-Ref-src/alphafold3-official/src/alphafold3/model/network/diffusion.py`

## Overview

### Purpose
- Encodes the current noise level t ∈ [0, 1] into a vector
- Allows the network to know how much noise is present
- Enables conditioning of all layers on noise level

### Key Properties
- Sinusoidal: Uses sin/cos of multiple frequencies
- Continuous: Smooth representation for any t
- Unique: Each noise level gets a distinct embedding

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(42)

In [ ]:
def noise_level_embedding(t, c=256, max_period=10000):
    """
    Sinusoidal noise level embedding.
    
    Args:
        t: Noise level (scalar or array), typically in [0, 1]
        c: Embedding dimension
        max_period: Maximum period for lowest frequency
    
    Returns:
        Embedding vector [c] or [batch, c]
    """
    t = np.atleast_1d(t)
    half = c // 2
    
    # Compute frequencies
    freqs = np.exp(-np.log(max_period) * np.arange(half) / half)
    
    # Compute arguments
    args = t[:, None] * freqs[None, :]  # [batch, half]
    
    # Sinusoidal embedding
    emb = np.concatenate([np.cos(args), np.sin(args)], axis=-1)  # [batch, c]
    
    return emb.squeeze()

In [ ]:
def noise_level_embedding_mlp(t, c=256, hidden=512):
    """
    Noise level embedding with MLP projection.
    
    Often the sinusoidal embedding is passed through an MLP
    for additional expressivity.
    """
    # Base sinusoidal embedding
    emb = noise_level_embedding(t, c)
    
    # MLP: Linear -> SiLU -> Linear
    W1 = np.random.randn(c, hidden) * (c ** -0.5)
    W2 = np.random.randn(hidden, c) * (hidden ** -0.5)
    
    h = emb @ W1
    h = h / (1 + np.exp(-h))  # SiLU/Swish
    out = h @ W2
    
    return out

In [ ]:
# Test: Embedding properties
print("Test: Noise Level Embedding")
print("="*60)

# Single embedding
t = 0.5
emb = noise_level_embedding(t, c=64)
print(f"Single embedding shape: {emb.shape}")
print(f"Embedding norm: {np.linalg.norm(emb):.4f}")

# Batch embedding
t_batch = np.array([0.0, 0.25, 0.5, 0.75, 1.0])
emb_batch = noise_level_embedding(t_batch, c=64)
print(f"\nBatch embedding shape: {emb_batch.shape}")

# Check distinctness
print("\nCosine similarity between different t:")
for i in range(len(t_batch)):
    for j in range(i+1, len(t_batch)):
        sim = np.dot(emb_batch[i], emb_batch[j]) / (np.linalg.norm(emb_batch[i]) * np.linalg.norm(emb_batch[j]))
        print(f"  t={t_batch[i]:.2f} vs t={t_batch[j]:.2f}: {sim:.4f}")

In [ ]:
# Visualize embeddings
print("\nVisualization: Noise Level Embeddings")
print("="*60)

t_vals = np.linspace(0, 1, 100)
embeddings = noise_level_embedding(t_vals, c=64)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# First few dimensions
ax = axes[0]
for d in range(8):
    ax.plot(t_vals, embeddings[:, d], label=f'dim {d}')
ax.set_xlabel('Noise level t')
ax.set_ylabel('Embedding value')
ax.set_title('First 8 embedding dimensions')
ax.legend()

# Similarity matrix
ax = axes[1]
t_sample = np.linspace(0, 1, 20)
emb_sample = noise_level_embedding(t_sample, c=64)
sim_matrix = emb_sample @ emb_sample.T
sim_matrix = sim_matrix / np.outer(np.linalg.norm(emb_sample, axis=1), np.linalg.norm(emb_sample, axis=1))
im = ax.imshow(sim_matrix, cmap='viridis')
ax.set_xlabel('t index')
ax.set_ylabel('t index')
ax.set_title('Cosine similarity matrix')
plt.colorbar(im, ax=ax)

plt.tight_layout()
plt.savefig('/tmp/noise_embedding.png', dpi=100)
plt.close()
print("Saved visualization to /tmp/noise_embedding.png")

## Key Insights

1. **Sinusoidal**: Uses sin/cos at multiple frequencies
2. **Smooth Interpolation**: Nearby t values have similar embeddings
3. **Distinct**: Each noise level is uniquely represented
4. **Conditioning**: Used throughout diffusion transformer via AdaLN